# Crop and Pair


This code crop the grey level zone, hide the yellow letters and pair the images original-GT

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import skimage.transform as skt
from skimage.restoration import inpaint
import os
import cv2
from scipy.signal import fftconvolve

In [2]:
def amagar_colors(im):
    """ Hide de yellow letters and return a black and white image"""
    im = 255*im if np.max(im)<1.1 else im
    #select the black and white pixels (same value in the three channels)
    mask1=im[:,:,0]==im[:,:,1] 
    mask2=im[:,:,0]==im[:,:,2] 
    mask=mask1+mask2
    
    #the new image will be only black and white
    removed=im[:,:,0]*mask
    
    #invert the mask
    mask=abs(mask-1)

    # increase a little bit the mask to be sure to hide the colors
    mask=fftconvolve(mask,np.array([[0,1,0],[1,1,1],[0,1,0]]),mode='same')
    
    #inpaint the black and white image
    final=cv2.inpaint(removed,np.asarray(255*mask,dtype='uint8'),1,cv2.INPAINT_TELEA)

    return(final)

In [3]:
def crop(im,tipus):
    """
    Crop the image depending on their type
    tipus:
    -original
    -modificada
    """
    if tipus=='original':
        if im.shape==(422, 636, 4):
            im1_crop_man=amagar_colors(im[:-57,129:-116,:])/255.
            return im1_crop_man
        else:
            dire='wrong_shape/originals/'
            os.makedirs(dire)
            plt.imsave(dire+str(len(os.listdir(dire))))
            return 'Not desidered shape',im.shape
    elif tipus=='modificada':
        if im.shape==(434, 636, 4):
            im2_crop_man=(im[22:-92,152:-140,0:3])
            #the output pixels are in 0-1 range
            im2_resize=skt.resize(im2_crop_man,(365, 391, 3))
            im2_array=np.asanyarray(im2_resize) 
            return im2_array
        else:
            dire='wrong_shape/modificades/'
            os.makedirs(dire)
            plt.imsave(dire+str(len(os.listdir(dire))))

            return 'Not desidered shape'
        return 'wrong type'

In [4]:
def normalize(arr):
    rng = arr.max()-arr.min()
    amin = arr.min()
    return (arr-amin)/rng

In [5]:
def pair_images(nova_ruta,desti='/'):
    """
       For each ITM or PL image look for the original image that is more simmilar and pair them.
    
        nova_ruta: carpeta de cada pacient (amb la barra al final)
        desti: lloc on es crearan les carpetes amb les parelles ja tallades i arreglades (amb barra al final)"""
    
    # store the images to the diccionary with a number as a key
    originals={}
    try:
        num=0
        for im in os.listdir(nova_ruta+'originals'):
            a=plt.imread(nova_ruta+'originals/'+im)
            #crop the original image
            originals[str(num)]=crop(a,'original')
            num+=1
    except:
        return
    imts={}
    try:
        for im in os.listdir(nova_ruta+'IMT'):
            #crop the image
            imITM=crop(plt.imread(nova_ruta+'IMT/'+im),'modificada')
            differences=[]
            for index in range(len(originals)):
                b=((originals[str(index)]))
                #find the differences with diverse candidates
                diff=np.sum(np.abs(normalize(imITM[:,:,0])-normalize(originals[str(index)])))
                differences.append(diff)
            imts[str(np.argmin(differences))]=imITM
    except:
        imts={}

    # repeat the same with the plaque
    pls={}
    try:
        for im in os.listdir(nova_ruta+'PL'):
            imPL=crop(plt.imread(nova_ruta+'PL/'+im),'modificada')
            differences=[]
            for index in range(len(originals)):
                b=((originals[str(index)]))
                diff=np.sum(np.abs(normalize(imPL[:,:,0])-normalize(originals[str(index)])))
                differences.append(diff)
            pls[str(np.argmin(differences))]=imPL
    except:
        pls={}
        
    split=nova_ruta.split('/')
    pacient=split[-2]+'_'+split[-3]
    
    #creates the destinations folders
    ruta_itms=desti+'parelles_imt/'
    os.makedirs(ruta_itms+'IMT/')if not os.path.exists(ruta_itms+'IMT/') else None
    os.makedirs(ruta_itms+'original/')if not os.path.exists(ruta_itms+'original/') else None
    os.makedirs(ruta_itms+'IMT_rar/')if not os.path.exists(ruta_itms+'IMT_rar/') else None
    os.makedirs(ruta_itms+'original_rar/')if not os.path.exists(ruta_itms+'original_rar/') else None

    ruta_pls=desti+'parelles_pl/'
    os.makedirs(ruta_pls+'PL/')if not os.path.exists(ruta_pls+'PL/') else None
    os.makedirs(ruta_pls+'original/')if not os.path.exists(ruta_pls+'original/') else None
    os.makedirs(ruta_pls+'PL_rar/')if not os.path.exists(ruta_pls+'PL_rar/') else None
    os.makedirs(ruta_pls+'original_rar/')if not os.path.exists(ruta_pls+'original_rar/') else None

    #match the indices of the originals and imt/pl
    for index in originals.keys():
        if index in imts.keys():
            
            #split some images where the grey zone is smaller so they have more blacks in the first columns
            if np.sum(imts[index][:,0:10,0])<100:
                
                plt.imsave(ruta_itms+'IMT_rar/'+pacient+'_'+index+'.png',imts[index])
                plt.imsave(ruta_itms+'original_rar/'+pacient+'_'+index+'.png',originals[index],cmap='gray')
            else:
                plt.imsave(ruta_itms+'IMT/'+pacient+'_'+index+'.png',imts[index])
                plt.imsave(ruta_itms+'original/'+pacient+'_'+index+'.png',originals[index],cmap='gray')

    for index in originals.keys():
        if index in pls.keys():
            if np.sum(pls[index][:,0:10,0])<100:
                plt.imsave(ruta_pls+'PL_rar/'+pacient+'_'+index+'.png',pls[index])
                plt.imsave(ruta_pls+'original_rar/'+pacient+'_'+index+'.png',originals[index],cmap='gray')
            else:
                plt.imsave(ruta_pls+'PL/'+pacient+'_'+index+'.png',pls[index])
                plt.imsave(ruta_pls+'original/'+pacient+'_'+index+'.png',originals[index],cmap='gray')

In [4]:
ruta='/media/HDD3TB/data_carotid/NEFRONA_parts/ACC/'
llista=os.listdir(ruta)
for banda in os.listdir(ruta):
    for pacient in os.listdir(ruta+'/'+banda):
        pair_images(ruta+'/'+banda+'/'+pacient+'/',desti='/media/HDD3TB/data_carotid/NEFRONA_crops/')